In [1]:
import numpy as np
import time
x_train=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist30kfromsenet.npy')
x_train=x_train.reshape(30000,2000)
y=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist30kfromsenet.npy')
y=y.reshape(30000,)
alpha_trans1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/cknnmnist30kfromsenet.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 
print(x_train.shape,y.shape,alpha_trans1.shape)

(30000, 2000) (30000,) (60, 500, 500)


In [2]:
!pip install memory_profiler
%load_ext memory_profiler

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%memit

peak memory: 351.12 MiB, increment: 0.03 MiB


In [3]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=500
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 10
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres

from numpy import save


def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W

def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    #print('uc1 shape is',l1)
    #print('uc2 shape is',l2)
    
    ## if 10==10 return true else return false
    assert(l1 == l2 and np.all(uc1 == uc2))
    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes) 
    m = Munkres()
    indexes = m.compute(cost_matrix)
    mapper = { old: new for (old, new) in indexes }
    new_labels = translate_clustering(labels, mapper)
    new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
    return new_labels,accuracy(new_cm)


def arch():
    encoding_dim = 490 
    input_img = Input(shape=(2000,))
    encoded_layer_2 = Dense(1500, activation='tanh')(input_img)
    encoded_layer_3 = Dense(1000, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(700, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(700, activation='tanh')(encoded)
    decoded_layer_3 = Dense(1000, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(1500, activation='tanh')(decoded_layer_3)
    decoded = Dense(2000, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g][:,:])
        return structpreserve+l1norm+l2norm+orthonormal
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt))))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4

def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [5]:
nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)

In [6]:
%memit

peak memory: 752.20 MiB, increment: 1.60 MiB


In [7]:
startae = time.time()
encoder,encoding_dim,autoencoder=arch()
autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist30k.ckpt')
endae=time.time()
print('AE training time is : ',endae - startae)

print('training from scratch')
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist30k.ckpt')
g=0
encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())
%memit


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 30000 samples
Epoch 1/100
30000/30000 [==============================] - 25s 825us/sample - loss: 3.0943e-04
Epoch 2/100
30000/30000 [==============================] - 25s 833us/sample - loss: 1.1976e-04
Epoch 3/100
30000/30000 [==============================] - 23s 768us/sample - loss: 8.8512e-05
Epoch 4/100
30000/30000 [==============================] - 25s 817us/sample - loss: 7.3116e-05
Epoch 5/100
30000/30000 [==============================] - 24s 795us/sample - loss: 6.3577e-05
Epoch 6/100
30000/30000 [==============================] - 24s 792us/sample - loss: 5.6960e-05
Epoch 7/100
30000/30000 [==============================] - 24s 786us/sample - loss: 5.2011e-05
Epoch 8/100
30000/30000 [==============================] - 25s 818us/sample - loss: 4.8141e-05
Epoch 9/100
30000/30000 [==============================] - 23s 760us/sample - loss: 4.5004e-05
Epoch 10/100
30000/30000 [===============

30000/30000 [==============================] - 33s 1ms/sample - loss: 1.6168e-05
Epoch 83/100
30000/30000 [==============================] - 27s 911us/sample - loss: 1.6120e-05
Epoch 84/100
30000/30000 [==============================] - 28s 924us/sample - loss: 1.6047e-05
Epoch 85/100
30000/30000 [==============================] - 28s 936us/sample - loss: 1.6006e-05
Epoch 86/100
30000/30000 [==============================] - 27s 911us/sample - loss: 1.5938e-05
Epoch 87/100
30000/30000 [==============================] - 27s 909us/sample - loss: 1.5876e-05
Epoch 88/100
30000/30000 [==============================] - 30s 991us/sample - loss: 1.5827e-05
Epoch 89/100
30000/30000 [==============================] - 28s 940us/sample - loss: 1.5768e-05
Epoch 90/100
30000/30000 [==============================] - 28s 943us/sample - loss: 1.5717e-05
Epoch 91/100
30000/30000 [==============================] - 30s 997us/sample - loss: 1.5670e-05
Epoch 92/100
30000/30000 [=============================

In [8]:
start = time.time()
maxnum=0
for t in range (1,300,1):
    g=0
    for l in range (0,len(x_train),batchSize):
        encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
        g=g+1
    z_test=encoder.predict(x_train)
    k_means = KMeans(n_clusters=k_cluster, init='k-means++')
    ic= k_means.fit(z_test).cluster_centers_ 
    assignment = assign_func(z_test, ic)
    predictedlabels = np.argmax(assignment, axis=1)
    rectified_label, new_acc=rectify_label(predictedlabels, y)
    print('acc at',t, 'epoch is:',maxnum)
    if maxnum<new_acc:
        maxnum=new_acc
        encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist30k.ckpt')
        np.save('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist30k.npy',ic)
        maxnmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
        maxf1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
        maxprec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
    else:
        continue
end=time.time()
print('E training time is : ',end - start)

Train on 500 samples
500/500 [==============================] - 22s 44ms/sample - loss: 1.4127
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4360
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4296
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4061
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4137
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4175
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4196
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4135
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.4232
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4283
Train on 500 samples
500/500 [==============================] - 1s 2

Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4001
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.4213
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3996
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3994
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3999
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.4011
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3945
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3964
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3941
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3886
Train on 500 samples
500/500 [==============================] - 1s 2ms

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3831
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3783
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3812
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3721
acc at 3 epoch is: 0.5592666666666667
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3639
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3861
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3839
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3595
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3691
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3699
Train on 500 samples
500/500 [==

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3572
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3618
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3690
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3521
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3575
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3743
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3580
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3561
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3584
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3585
Train on 500 samples
500/500 [==============================] - 1s 1ms

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3455
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3475
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3517
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3399
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3457
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3386
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3433
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3360
acc at 6 epoch is: 0.6291666666666667
Train on 500 samples
500/500 [==============================] - 1s 2ms/sample - loss: 1.3247
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.3463
Train on 500 samples
500/500 [==

...

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0012
Train on 500 samples
500/500 [==============================] - 0s 678us/sample - loss: 1.0253
Train on 500 samples
500/500 [==============================] - 0s 810us/sample - loss: 0.9827
Train on 500 samples
500/500 [==============================] - 0s 844us/sample - loss: 1.0070
Train on 500 samples
500/500 [==============================] - 0s 848us/sample - loss: 0.9948
Train on 500 samples
500/500 [==============================] - 0s 870us/sample - loss: 0.9944
Train on 500 samples
500/500 [==============================] - 0s 714us/sample - loss: 0.9989
Train on 500 samples
500/500 [==============================] - 0s 962us/sample - loss: 0.9980
Train on 500 samples
500/500 [==============================] - 0s 870us/sample - loss: 0.9992
Train on 500 samples
500/500 [==============================] - 0s 833us/sample - loss: 0.9945
Train on 500 samples
500/500 [======================

500/500 [==============================] - 0s 707us/sample - loss: 0.9951
Train on 500 samples
500/500 [==============================] - 0s 814us/sample - loss: 1.0066
Train on 500 samples
500/500 [==============================] - 0s 745us/sample - loss: 1.0047
Train on 500 samples
500/500 [==============================] - 0s 792us/sample - loss: 0.9998
Train on 500 samples
500/500 [==============================] - 0s 763us/sample - loss: 0.9967
Train on 500 samples
500/500 [==============================] - 0s 854us/sample - loss: 1.0041
Train on 500 samples
500/500 [==============================] - 0s 702us/sample - loss: 1.0016
Train on 500 samples
500/500 [==============================] - 0s 771us/sample - loss: 1.0038
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0122
Train on 500 samples
500/500 [==============================] - 0s 944us/sample - loss: 1.0040
Train on 500 samples
500/500 [==============================] - 0s 943us/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Error in sys.excepthook:
Traceback (most recent call last):
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2047, in showtraceback
    value, tb, tb_offset=tb_offset)
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1436, in structured_traceback
    self, etype, value, tb, tb_offset, number_of_lines_of_context)
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1336, in structured_traceback
    self, etype, value, tb, tb_o

Traceback (most recent call last):
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 316, in wrapped
    return f(*args, **kwargs)
  File "/home/itu-cvl/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 350, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python3.6/inspect.py", line 1490, in getinnerframes
    frameinfo = (tb.tb_frame,) + ge

KeyboardInterrupt: 

In [9]:
%memit

peak memory: 2779.90 MiB, increment: 0.50 MiB


In [10]:
print(maxnum,maxnmi,maxf1,maxprec)

0.9643333333333334 0.9083955277415052 0.9643333333333334 0.9644723278321462


In [4]:
import numpy as np
import time
x1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist70kfromsenet.npy')
x1=x1.reshape(70000,2000)
x_test=x1[30000:,:]
y1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist70kfromsenet.npy')
y1=y1.reshape(70000,)
y_test=y1[30000:,]

start=time.time()
encoder,encoding_dim,autoencoder=arch()
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist30k.ckpt')
ic=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist30k.npy')
z_test=encoder.predict(x_test)
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, predictedlabels)
f1=sklearn.metrics.f1_score(y_test, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print(acc,nmi,f1,prec)
end=time.time()
print('time is',end-start)
%memit

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
0.96655 0.9132669519402965 0.96655 0.9666804380239661
time is 9.156223773956299
peak memory: 1454.01 MiB, increment: -0.05 MiB
